In [43]:
# Helpers
import numpy as np

# DB 
import psycopg2
from django.conf import settings

# Evaluation
from sklearn.metrics import roc_curve, auc, f1_score
from scipy import special

# Plot
import matplotlib.pyplot as plt
import seaborn as sns

In [44]:
connection = psycopg2.connect(user = settings.DATABASES["default"]["USER"],
                                  password = settings.DATABASES["default"]["PASSWORD"],
                                  host = settings.DATABASES["default"]["HOST"],
                                  port = settings.DATABASES["default"]["PORT"],
                                  database = settings.DATABASES["default"]["NAME"])
connection.autocommit=True
cursor = connection.cursor()

In [45]:
%%time
query = """select experiment_id, sp, x, sp_hat from EDM2020_2020_02_19 where method='fdtf'"""
cursor.execute(query)
row = cursor.fetchone()

CPU times: user 16 s, sys: 16.2 s, total: 32.2 s
Wall time: 14min 6s


In [46]:
%%time
update_queries = []
total = 0
error_count = 0
while row:
    exp_id, sp, X, sp_hat = row
    sp = np.asarray(sp)
    X = np.asarray(X)
    sp_hat = np.asarray(sp_hat)
    
    attempts_per_question = X.sum(axis=0).sum(axis=1)
    del_questions = [idx for idx,value in enumerate(attempts_per_question) if value < 3]
    X = np.delete(X, del_questions, axis=1)
    
    # Get train predicted values
    y = sp[np.where(X == 1)]
    y_pred = sp_hat[np.where(X == 1)]
    
    # Calculate AUC and threshold to binarize prediction
    fpr, tpr, thresholds = roc_curve(y, y_pred, pos_label=1)
    auc_score = auc(fpr, tpr)
    J_stats = tpr - fpr
    J_opt_thresholds = thresholds[np.argmax(J_stats)]
    y_pred_binary = np.where(np.asarray(y_pred) > J_opt_thresholds, 1, 0)
    
    # Calculate accuracy, RMSE, F1 and NLL
    acc = np.logical_not(np.logical_xor(y, y_pred_binary)).sum()/len(y)
    rmse = np.sqrt(np.mean(np.power(y - y_pred, 2))) 
    f1 = f1_score(y, y_pred_binary)
    nll = -special.xlogy(y, y_pred) - special.xlogy(1-y, 1-y_pred)

    y_pred_all = np.zeros((y.shape[0], 2))
    y_pred_all[:,0] = 1-y_pred
    y_pred_all[:,1] = y_pred
    nll_special = (-np.ma.log(y_pred_all[range(y.shape[0]),y.astype(int)])).mean()

    # Substitute inf by max NLL
    idx = np.where(np.isinf(nll))
    nll[idx] = 0
    max_nll = np.max(nll)
    nll[idx] = max_nll
    nll = np.mean(nll)
    
    update = {
        "train_auc": auc_score,
        "train_acc": acc,
        "train_rmse": rmse,
        "auc_threshold": J_opt_thresholds,
        "train_1":  np.where(y == 1)[0].shape[0],
        "train_0":  np.where(y == 0)[0].shape[0],
        "train_pred_1": np.where(y_pred_binary == 1)[0].shape[0],
        "train_pred_0": np.where(y_pred_binary == 0)[0].shape[0],
        "train_f1": f1,
        "train_nll": nll,
        "train_nll_special": nll_special,
        "train_nll_masked": idx[0].shape[0]
    }
    
    
    # Write PSQL query
    update_query = "UPDATE EDM2020_2020_02_19 SET "
    update_list = []
    query_values = []
    for key, value in update.items():
        update_list.append(key + "= %s")
        query_values.append(value)
    
    update_query += ", ".join(update_list) + "where experiment_id = %s"
    query_values.append(exp_id)
    query_values = tuple(query_values)
    query = cursor.mogrify(update_query, query_values)
    update_queries.append(query)
    total += 1
    try:
        row = cursor.fetchone()
    except psycopg2.ProgrammingError:
        row = False

CPU times: user 4min 23s, sys: 21.3 s, total: 4min 44s
Wall time: 4min 44s


In [47]:
len(update_queries)

763

In [49]:
for query in update_queries:
    cursor.execute(query)